In [1]:
from dotenv import find_dotenv,load_dotenv; _=load_dotenv(find_dotenv())

In [2]:
import json, os, datetime, requests
from tzlocal import get_localzone
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from pydantic import BaseModel, Field
from typing import Union, Any, Optional, Type
from enum import Enum
import googlemaps; gmaps = googlemaps.Client(key=os.environ['GOOGLE_MAPS_API_KEY'])

In [3]:
# gmaps.reverse_geocode(
#     latlng={"latitude": "35.1796","longitude": "136.9166"},
#     language='ja'
# )

In [4]:
def get_now():
    now = datetime.datetime.now().replace(tzinfo=get_localzone())
    return now.strftime('%Y-%m-%d %H:%M %z')

get_now_tool_description = """\
You can use to get current datetime with no input. You can use also to use get current date.
"""

print(get_now())

get_current_datetime_tool = StructuredTool.from_function(
    func = get_now,
    # name='CurrentLocationGetter',
    description=get_now_tool_description,
)

2023-08-30 11:41 +0900


In [5]:
def get_current_location():
    geo_request_url = 'https://get.geojs.io/v1/ip/geo.json'
    data = requests.get(geo_request_url).json()
    return {
        'latitude': data['latitude'],
        'longitude': data['longitude'],
    }
print(json.dumps(get_current_location(),indent=2,ensure_ascii=False))

get_current_location_tool_description = """\
You can get current location with no input.
"""

get_current_location_tool = StructuredTool.from_function(
    func = get_current_location,
    # name='CurrentLocationGetter',
    description=get_current_location_tool_description,
)

{
  "latitude": "35.4643",
  "longitude": "139.6262"
}


In [6]:
def simplified_place_dict(res):
    r = res[0]
    return {
        'address': r['formatted_address'],
        'location': r['geometry']['location'],
        'types': r['types']
    }

def find_place_with_text_func(query: str):
    places_result = gmaps.find_place(query,input_type='textquery')
    geocode_results = [
        gmaps.geocode(place_id=r['place_id'])
        for r in places_result['candidates']
    ]
    # print(json.dumps(geocode_results,indent=2,ensure_ascii=False))
    return [simplified_place_dict(r) for r in geocode_results]
# print(find_place_with_text_func(query='ガスト港東通店'))
find_place_tool_description = """\
A Find Place request takes a text input, and returns a place.
The text input can be any kind of Places data, for example,
a name, address, or phone number.
"""

find_place_tool = Tool.from_function(
    func = find_place_with_text_func,
    name='PlaceSearch',
    description=find_place_tool_description,
)

In [7]:
# import datetime,pytz
# print(datetime.datetime.strptime('2023/8/30 09:00','%Y/%m/%d %H:%M').astimezone(tz=datetime.timezone.utc).timestamp())
# print(datetime.datetime.strptime('2023/8/30 09:00','%Y/%m/%d %H:%M').astimezone(pytz.timezone('Asia/Tokyo')).timestamp())

In [8]:
# import datetime

# # 日本の時刻を設定（例：2023年8月30日 15:00 JST）
# japan_time = datetime.datetime(2023, 8, 30, 15, 0, tzinfo=datetime.timezone(datetime.timedelta(hours=9)))

# # 日本標準時の場合
# utc_time_standard = japan_time - datetime.timedelta(hours=9)
# utc_timestamp_standard = utc_time_standard.timestamp()

# print("UTCタイムスタンプ（日本標準時）:", utc_timestamp_standard)

In [9]:
timestamp = 1693440000
dt_object = datetime.datetime.fromtimestamp(timestamp)#,tz=datetime.timezone.utc)
dt_object = dt_object.astimezone(tz=datetime.timezone.utc)

print(dt_object)

2023-08-31 00:00:00+00:00


In [10]:
from tzlocal import get_localzone
tz = get_localzone()
 
print(tz)

Asia/Tokyo


In [11]:
class Mode(Enum):
    DRIVING = "driving"
    WALKING = "walking"
    BYCYCLING = "bicycling"
    TRANSIT = "transit"

class DirectionsInput(BaseModel):
    origin: Union[str,dict] = Field(
        description='The address in string or latitude/longitude in dict from which you wish to calculate directions.'
    )
    destination: Union[str,dict] = Field(
        description='The address in string or latitude/longitude in dict from which you wish to calculate directions.'
    )
    mode: Mode = Field(
        description='Specifies the mode of transport to use when calculating directions.'
    )
    departure_time: Optional[str] = Field(
        description="Specifies the desired time of departure. Note: you can't specify both departure_time and arrival_time. The format must be \"%Y/%m/%d %H:%M %z\"."
    )
    arrival_time: Optional[str] = Field(
        description="Specifies the desired time of arrival for transit directions. Note: you can't specify both departure_time and arrival_time. The format must be \"%Y/%m/%d %H:%M %z\"."
    )

def simplified_directions_dict(res):
    r = res['legs'][0]
    return {
        'arrival_time': r['arrival_time']['text'] if 'arrival_time' in r.keys() else None,
        'departure_time': r['departure_time']['text'] if 'departure_time' in r.keys() else None,
        'distance': r['distance']['text'] if 'distance' in r.keys() else None,
        'duration': r['duration']['text'] if 'duration' in r.keys() else None,
        'steps': [
            {
                'distance': s['distance']['text'],
                'duration': s['duration']['text'],
                'travel_mode': s['travel_mode'],
                "instruction": s['html_instructions'],
                'transit_details': {
                    'arrival_stop': s['transit_details']['arrival_stop'],
                    'departure_stop': s['transit_details']['departure_stop'],

                } if 'transit_details' in s.keys() else None
            }
            for s in r['steps']
        ]
    }

# def get_direction_func(params: DirectionsInput):
#     places_result = gmaps.directions_result = gmaps.directions(
#         origin=params.origin,
#         destination=params.destination,
#         mode=params.mode.value,
#     )
#     # print(json.dumps(geocode_results,indent=2,ensure_ascii=False))
#     return [simplified_directions_dict(r) for r in places_result]

directions_tool_description = """\
Get directions between an origin point and a destination point.
"""

# get_direction_tool = Tool.from_function(
#     func = get_direction_func,
#     name='Directions',
#     description=directions_tool_description,
# )

class DirectionsTool(BaseTool):
    name = "Directions"
    description = directions_tool_description
    args_schema: Type[DirectionsInput] = DirectionsInput

    def _run(self, origin, destination, mode, departure_time=None, arrival_time=None):
        print(self.datetime_to_timestamp(departure_time))
        directions_result = gmaps.directions(
            origin=origin,
            destination=destination,
            mode=mode.value,
            departure_time=self.datetime_to_timestamp(departure_time),
            arrival_time=self.datetime_to_timestamp(arrival_time)
        )
        # print(json.dumps(geocode_results,indent=2,ensure_ascii=False))
        return [simplified_directions_dict(r) for r in directions_result]

    async def _arun(self, origin, destination, mode):
        return self._run(origin, destination, mode)

    def datetime_to_timestamp(self,datetime_str):
        if datetime_str is None:
            return None
        dt = self.create_datetime(datetime_str) # datetime.datetime.strptime(datetime_str,'%Y-%m-%d %H:%M %z')
        # dt = dt.astimezone(get_localzone())
        return int(dt.timestamp())
    
    @staticmethod
    def create_datetime(input_string):
        formats = ["%Y/%m/%d %H:%M %z", "%Y-%m-%d %H:%M %z"]
        for fmt in formats:
            try:
                dt = datetime.datetime.strptime(input_string, fmt)
                return dt
            except ValueError:
                pass
        raise ValueError("Input string does not match any supported format")

get_direction_tool = DirectionsTool()

print(get_direction_tool._run(
    origin='Tower of London',
    destination='Backingum Palace',
    # origin={'latitude': '35.1796', 'longitude': '136.9166'},
    # destination={'latitude': '35.091708', 'longitude': '136.916568'},
    mode=Mode.TRANSIT,
    departure_time='2023-8-30 09:00 +0100'
))

1693382400
[{'arrival_time': '9:28\u202fAM', 'departure_time': '9:01\u202fAM', 'distance': '5.9 km', 'duration': '27 mins', 'steps': [{'distance': '0.4 km', 'duration': '6 mins', 'travel_mode': 'WALKING', 'instruction': 'Walk to Tower Hill', 'transit_details': None}, {'distance': '4.8 km', 'duration': '12 mins', 'travel_mode': 'TRANSIT', 'instruction': 'Subway towards Ealing Broadway', 'transit_details': {'arrival_stop': {'location': {'lat': 51.499325, 'lng': -0.134587}, 'name': "St. James' Park"}, 'departure_stop': {'location': {'lat': 51.509964, 'lng': -0.07672799999999999}, 'name': 'Tower Hill'}}}, {'distance': '0.7 km', 'duration': '9 mins', 'travel_mode': 'WALKING', 'instruction': 'Walk to London SW1A 1AA, UK', 'transit_details': None}]}]


In [12]:
llm = ChatOpenAI(temperature=0)

In [13]:
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
}
memory = ConversationBufferMemory(memory_key="memory", return_messages=True)

In [14]:
tools = [
    get_current_datetime_tool,
    get_current_location_tool,
    find_place_tool,
    get_direction_tool
]
agent = initialize_agent(
    tools, llm, agent=AgentType.OPENAI_FUNCTIONS,
    memory=memory, agent_kwargs=agent_kwargs,
    verbose=True
)

In [15]:
input_text = 'Tower of Londonはどこにありますか？'
agent.run(input_text)



> Entering new AgentExecutor chain...

Invoking: `PlaceSearch` with `Tower of London`


[{'address': '4-chōme-23-9 Meieki, Nakamura Ward, Nagoya, Aichi 450-0002, Japan', 'location': {'lat': 35.168371, 'lng': 136.889019}, 'types': ['bar', 'establishment', 'food', 'point_of_interest', 'restaurant']}, {'address': 'London EC3N 4AB, UK', 'location': {'lat': 51.50811239999999, 'lng': -0.0759493}, 'types': ['establishment', 'landmark', 'point_of_interest', 'tourist_attraction']}]Tower of Londonは、イギリスのロンドンにあります。具体的な住所はLondon EC3N 4ABです。

> Finished chain.


'Tower of Londonは、イギリスのロンドンにあります。具体的な住所はLondon EC3N 4ABです。'

In [16]:
input_text = 'バッキンガム宮殿からの行き方を教えてください。'
# input_text = 'エスターテ南青山から愛知県名古屋市南区港東通１丁目２４まで徒歩何分ですか？'
agent.run(input_text)



> Entering new AgentExecutor chain...

Invoking: `PlaceSearch` with `バッキンガム宮殿`


[{'address': 'Japan, 〒155-0031 Tokyo, Setagaya City, Kitazawa, 1-chōme−45−１５−バッキンガム 宮殿Suzunari', 'location': {'lat': 35.662442, 'lng': 139.6703417}, 'types': ['bar', 'establishment', 'food', 'point_of_interest', 'restaurant']}]
Invoking: `Directions` with `{'origin': 'バッキンガム宮殿', 'destination': 'Tower of London', 'mode': 'transit'}`


None
[{'arrival_time': '4:34\u202fAM', 'departure_time': '3:52\u202fAM', 'distance': '6.4 km', 'duration': '42 mins', 'steps': [{'distance': '1.3 km', 'duration': '17 mins', 'travel_mode': 'WALKING', 'instruction': 'Walk to Piccadilly Circus (Stop F)', 'transit_details': None}, {'distance': '4.8 km', 'duration': '20 mins', 'travel_mode': 'TRANSIT', 'instruction': 'Bus towards Romford', 'transit_details': {'arrival_stop': {'location': {'lat': 51.5096953, 'lng': -0.0757877}, 'name': 'Tower of London (Stop TB)'}, 'departure_stop': {'location': {'lat': 51.5099563, 'lng': -0.1366

'バッキンガム宮殿からTower of Londonへの行き方は以下の通りです：\n\n1. バッキンガム宮殿からピカデリーサーカス（Stop F）まで徒歩で移動します（距離：1.3 km、所要時間：17分）。\n2. ピカデリーサーカス（Stop F）からロンフォード行きのバスに乗ります（距離：4.8 km、所要時間：20分）。バスの詳細は以下の通りです：\n   - 出発地：ピカデリーサーカス（Stop F）\n   - 到着地：Tower of London（Stop TB）\n3. Tower of London（Stop TB）から目的地のLondon EC3N 4ABまで徒歩で移動します（距離：0.4 km、所要時間：5分）。\n\n所要時間は合計で42分です。到着予定時刻は午前4時34分です。'

In [17]:
input_text = '明日の朝9時に出発した場合の到着時刻を教えてください。'
agent.run(input_text)



> Entering new AgentExecutor chain...

Invoking: `Directions` with `{'origin': 'Buckingham Palace, London', 'destination': 'Tower of London, London', 'mode': 'transit', 'departure_time': '2022/12/01 09:00 +0000'}`


1669885200
[]申し訳ありませんが、現在の交通情報を取得できません。到着時刻を正確に計算するためには、実際の交通状況を考慮する必要があります。お手数ですが、出発の前日に再度お試しください。

> Finished chain.


'申し訳ありませんが、現在の交通情報を取得できません。到着時刻を正確に計算するためには、実際の交通状況を考慮する必要があります。お手数ですが、出発の前日に再度お試しください。'

In [18]:
input_text = 'ロンドン時間の明日9時の場合は？'
agent.run(input_text)



> Entering new AgentExecutor chain...

Invoking: `get_now` with `{}`


2023-08-30 11:41 +0900
Invoking: `Directions` with `{'origin': 'Buckingham Palace, London', 'destination': 'Tower of London, London EC3N 4AB', 'mode': 'transit', 'departure_time': '2023/08/31 09:00 +0100'}`


1693468800
[{'arrival_time': '9:27\u202fAM', 'departure_time': '9:00\u202fAM', 'distance': '5.9 km', 'duration': '27 mins', 'steps': [{'distance': '0.7 km', 'duration': '9 mins', 'travel_mode': 'WALKING', 'instruction': "Walk to St. James' Park", 'transit_details': None}, {'distance': '4.8 km', 'duration': '13 mins', 'travel_mode': 'TRANSIT', 'instruction': 'Subway towards Barking', 'transit_details': {'arrival_stop': {'location': {'lat': 51.510056, 'lng': -0.076782}, 'name': 'Tower Hill'}, 'departure_stop': {'location': {'lat': 51.499413, 'lng': -0.134587}, 'name': "St. James' Park"}}}, {'distance': '0.4 km', 'duration': '5 mins', 'travel_mode': 'WALKING', 'instruction': 'Walk to London EC3N 4AB, UK', 'trans

"明日の朝9時に出発した場合、Tower of Londonへの到着予定時刻は9時27分です。所要時間は27分で、距離は5.9 kmです。以下は詳細な経路です：\n\n1. バッキンガム宮殿からSt. James' Parkまで徒歩で移動します（距離：0.7 km、所要時間：9分）。\n2. St. James' ParkからBarking行きの地下鉄に乗ります（距離：4.8 km、所要時間：13分）。地下鉄の詳細は以下の通りです：\n   - 出発地：St. James' Park\n   - 到着地：Tower Hill\n3. Tower Hillから目的地のLondon EC3N 4ABまで徒歩で移動します（距離：0.4 km、所要時間：5分）。\n\nなお、この到着予定時刻は予測であり、実際の交通状況によって変動する可能性があります。ご参考までにお考えください。"

In [19]:
print(agent.memory.json(indent=2,ensure_ascii=False))

{
  "chat_memory": {
    "messages": [
      {
        "content": "Tower of Londonはどこにありますか？",
        "additional_kwargs": {},
        "example": false
      },
      {
        "content": "Tower of Londonは、イギリスのロンドンにあります。具体的な住所はLondon EC3N 4ABです。",
        "additional_kwargs": {},
        "example": false
      },
      {
        "content": "バッキンガム宮殿からの行き方を教えてください。",
        "additional_kwargs": {},
        "example": false
      },
      {
        "content": "バッキンガム宮殿からTower of Londonへの行き方は以下の通りです：\n\n1. バッキンガム宮殿からピカデリーサーカス（Stop F）まで徒歩で移動します（距離：1.3 km、所要時間：17分）。\n2. ピカデリーサーカス（Stop F）からロンフォード行きのバスに乗ります（距離：4.8 km、所要時間：20分）。バスの詳細は以下の通りです：\n   - 出発地：ピカデリーサーカス（Stop F）\n   - 到着地：Tower of London（Stop TB）\n3. Tower of London（Stop TB）から目的地のLondon EC3N 4ABまで徒歩で移動します（距離：0.4 km、所要時間：5分）。\n\n所要時間は合計で42分です。到着予定時刻は午前4時34分です。",
        "additional_kwargs": {},
        "example": false
      },
      {
        "content": "明日の朝9時に出発した場合の到着時刻を教えてください。",
        "additional_kwargs": {},
        "example